In [47]:
from cdapython import Q, columns, unique_terms, query


### Image files for a single subject

In [48]:
import json
q1 = Q('id = "TCGA-13-1504"')
r = q1.run()
print(r)


Getting results from database

Total execution time: 1386 ms

QueryID: b1f4bc35-95ff-43ba-9f95-8b9830f0e8a0
Query: SELECT all_v2.* FROM gdc-bq-sample.integration.all_v2 AS all_v2 WHERE (all_v2.id = 'TCGA-13-1504')
Offset: 0
Count: 1
Total Row Count: 1
More pages: False



In [52]:
from collections import Counter
categories = Counter()
imaging_types = Counter()
for f in r[0]['File']:
    dc = f['data_category']
    categories[dc] += 1
    if f['data_category'] == "Imaging":
        imaging_types[f['data_type']+'/'+f['file_format']] += 1
print(categories)
print("\nImaging type/format")
print(imaging_types)

Counter({'Imaging': 226, 'Peptide Spectral Matches': 55, 'Raw Mass Spectra': 28, 'Processed Mass Spectra': 27, 'Simple Nucleotide Variation': 16, 'Biospecimen': 12, 'Clinical': 8, 'Copy Number Variation': 7, 'Sequencing Reads': 3, 'Transcriptome Profiling': 2, 'DNA Methylation': 1, 'Proteome Profiling': 1})

Imaging type/format
Counter({'CT/DICOM': 226})


### Use DRS to get some details of each file
For illustration here's what DRS can tell us about one Imaging file

In [66]:
for f in r[0]['File'] :
    if f['data_category'] == "Imaging":
        drs_id = f['id']
        print(drs_id)
        break

f5027f67-011b-4f8e-a9bd-de73eaaf2fc0


In [67]:
from fasp.loc import crdcDRSClient
drsClient = crdcDRSClient()
drsClient.getObject(drs_id)

{'access_methods': [{'access_id': 'gs',
   'access_url': {'url': 'gs://idc-tcia-tcga-ov/dicom/1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018428492152614916489/1.3.6.1.4.1.14519.5.2.1.9203.4007.133576245722946348511732255420/1.3.6.1.4.1.14519.5.2.1.9203.4007.472644011742805252569503604125.dcm#1592638563565092'},
   'region': '',
   'type': 'gs'}],
 'aliases': [],
 'checksums': [{'checksum': '51f38e0cd236104a0a88a1a88b371155',
   'type': 'md5'}],
 'contents': [],
 'created_time': '2020-09-18T20:21:07.664086',
 'description': None,
 'form': 'object',
 'id': 'dg.4DFC/f5027f67-011b-4f8e-a9bd-de73eaaf2fc0',
 'mime_type': 'application/json',
 'name': None,
 'self_uri': 'drs://nci-crdc.datacommons.io/dg.4DFC/f5027f67-011b-4f8e-a9bd-de73eaaf2fc0',
 'size': 527958,
 'updated_time': '2020-09-18T20:21:07.664091',
 'version': 'e2be4764'}

To someone with some knowledge of dicom the access url field gives us a clue. This looks like a path to  nested directories named with dicom instance ids. The nesting of those directories reflects the dicom data model. That data model gives meaning to the files which is otherwise absent. In fact the full meaning is only within the files.

We can also see that there is a specific GCS bucket for the TCGA OV study (spoiler: study is overloaded, as we shall shortly discover :-)

Note the use of the word 'clue'. This is indeed detective work. Users should not have to do detective work.

In [54]:
import json
nn = 1
nmax = 1000
drs_list = []
for f in r[0]['File'] :
    if f['data_category'] == "Imaging":
        drs_id= 'crdc:'+f['id']
        print(drs_id)
        drs_list.append(mr.getObject(drs_id))
        nn += 1
    if nn > nmax:
        break
with open('drs_list.json', 'w') as outfile:
    json.dump(drs_list, outfile)

crdc:f5027f67-011b-4f8e-a9bd-de73eaaf2fc0
crdc:bd32819d-7ec5-49ee-800a-60b89c388cce
crdc:2a11facd-6043-4b8b-a746-5c66e4247c4b
crdc:9e0f0311-2fd7-46fc-8e53-6ee03596d81c
crdc:547cdc0a-c8b3-4fe4-a445-9ef71a6ce89a
crdc:87591dfe-2f8d-4fdc-812c-e719b09ce3f4
crdc:1f21ee2d-3414-4fd0-bce2-82ace64ffcef
crdc:95d03605-a111-4ace-8f99-0724e4124e97
crdc:018f936c-67b8-4750-900a-364bea5447f9
crdc:9280c959-4321-4b54-baea-cea72fa4cd87
crdc:41135e2f-349d-4362-8ad5-ccc0160e9c47
crdc:41846291-1236-4888-af28-ac421f325f3a
crdc:eca33c2a-b921-4091-a1dd-44d37486d921
crdc:d27da835-31b9-4622-88c9-58b8592118ed
crdc:8bb34822-1553-43b2-95a9-3b2b1ce2851e
crdc:74ebc760-d0bb-4668-bb81-db164a313add
crdc:16294e32-6bbf-4407-b01f-b62dac2561e7
crdc:2440372e-618f-494c-9f40-5d5c0a0be26b
crdc:a699ba7a-5991-44e7-9e83-5f6fc89995b5
crdc:6ff90873-e25a-42d7-b52c-7965414c3892
crdc:428a13ac-f50f-4634-8cd3-85cedf6f2800
crdc:9d039d61-3828-4df7-8263-83bc08819b7e
crdc:2ca61bc4-ea09-428a-a95f-246911a75189
crdc:42d1898a-03bc-4e2d-9c11-4d641

crdc:f7897efc-5282-46b7-afc3-1b3510c11438
crdc:e869dc79-89ab-4b99-8ff7-525a25ae9a62
crdc:3e28439d-a61b-45eb-858a-d67025e59157
crdc:9f37cef7-a80e-4753-97ed-8b6ff097b985
crdc:48a71fc5-c1bd-42ad-8d52-6e87b523c858
crdc:0faa0395-465f-4ecc-95ca-b1fe98fd1a62
crdc:76dcc698-d4ed-45bd-84a6-f440d6684329
crdc:3b7a4c27-30dd-4901-b7c9-f84f67d68e0c
crdc:4d486cbe-b166-4270-b477-74726ba85e2f
crdc:3627d545-9968-40c2-9207-4fd5be27f247
crdc:7379270f-3855-4539-a46f-a1e48b7a35e4
crdc:e5863f26-2ed9-4000-98d9-3fb590ccbea9
crdc:4c4b06b8-aad9-4c43-89dc-0a9cf983f8f1
crdc:db296715-279b-4907-909a-b434cdf02e11
crdc:093f50b4-3566-4a93-87c9-748f567bbefa
crdc:4c733655-32ba-4d7e-855d-bb543f847396
crdc:4edcf735-e61d-4795-b01a-d00e4a0ed951
crdc:3fbd40d2-de15-47eb-9de0-e2874f575702
crdc:b52d1ddf-15ca-4b85-8d23-a86efc940e30
crdc:fc5ce77e-e282-4da0-a0c2-e2b7bdd7e0e8
crdc:76829ecb-a6ce-4bca-9c29-0bdaa9c63b08
crdc:89cf6edd-1ca9-445c-8776-89241a469cee
crdc:a8644f45-e4c3-43fd-a15f-c1b25803f12a
crdc:60d5fc65-1754-4a1b-90ea-2c084

Write a tab delimited file containing the elements of the file path. i.e. extract the DICOM structure. If the path does not fit the pattern of 7 elements list it out.

In [56]:
import pandas as pd
dicom_lst = []
with open('dicom_list2.txt', 'w') as outfile:
    for d in drs_list:
        url = d['access_methods'][0]['access_url']['url']
        parts = url.split("/")
        if len(parts) == 7: 
            dicom_lst.append(parts[2:7])
        else:
            print(parts)
            print(json.dumps(d, indent=3))
        outstr = '\t'.join(parts)
        outfile.write(outstr+'\n')
        
df = pd.DataFrame(dicom_lst, columns =['bucket', 'folder', 'study', 'series', 'instance'])
df

['gs:', '', 'idc-open', '72eacf56-f87d-4af4-8128-47a6897d1506.dcm']
{
   "access_methods": [
      {
         "access_id": "gs",
         "access_url": {
            "url": "gs://idc-open/72eacf56-f87d-4af4-8128-47a6897d1506.dcm"
         },
         "region": "",
         "type": "gs"
      }
   ],
   "aliases": [],
   "checksums": [
      {
         "checksum": "da66bb0bccad300977e2c16762777868",
         "type": "md5"
      }
   ],
   "contents": [],
   "created_time": "2020-09-18T05:21:01.520509",
   "description": null,
   "form": "object",
   "id": "dg.4DFC/72eacf56-f87d-4af4-8128-47a6897d1506",
   "mime_type": "application/json",
   "name": null,
   "self_uri": "drs://nci-crdc.datacommons.io/dg.4DFC/72eacf56-f87d-4af4-8128-47a6897d1506",
   "size": 527968,
   "updated_time": "2021-11-15T19:04:09.450068",
   "version": "f2030c4c"
}


/Users/forei/imfenvs/cda2/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,bucket,folder,study,series,instance
0,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.133576245722...,1.3.6.1.4.1.14519.5.2.1.9203.4007.472644011742...
1,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.133576245722...,1.3.6.1.4.1.14519.5.2.1.9203.4007.203477636916...
2,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.148602785422...,1.3.6.1.4.1.14519.5.2.1.9203.4007.299208847780...
3,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.148602785422...,1.3.6.1.4.1.14519.5.2.1.9203.4007.308330171168...
4,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.148602785422...,1.3.6.1.4.1.14519.5.2.1.9203.4007.313603968394...
...,...,...,...,...,...
220,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.304277390592...,1.3.6.1.4.1.14519.5.2.1.9203.4007.797048602863...
221,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.133576245722...,1.3.6.1.4.1.14519.5.2.1.9203.4007.961053664838...
222,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.133576245722...,1.3.6.1.4.1.14519.5.2.1.9203.4007.354650146540...
223,idc-tcia-tcga-ov,dicom,1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018...,1.3.6.1.4.1.14519.5.2.1.9203.4007.304277390592...,1.3.6.1.4.1.14519.5.2.1.9203.4007.194304478572...


How many studies?

In [61]:
df.groupby('study').size()

study
1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018428492152614916489    225
dtype: int64

Just one study. So what series does it have within it, and how many instances (images) within each series?

In [62]:
df.groupby('series').size()

series
1.3.6.1.4.1.14519.5.2.1.9203.4007.133576245722946348511732255420     71
1.3.6.1.4.1.14519.5.2.1.9203.4007.148602785422578671396982235234     40
1.3.6.1.4.1.14519.5.2.1.9203.4007.304277390592235054246172081167    112
1.3.6.1.4.1.14519.5.2.1.9203.4007.327004617190337044238532022530      2
dtype: int64

There are 4 series each with widely varying numbers of instances within them. What a dicom instance is varies according to the imaging modality. The most common case is that an instance is a image.

What each series is is not clear from the ids above (btw these ids are dicom instance ids).

Note that we had to go through detective work to do this.


In [36]:
import requests
import os
def download(url, file_path):
    with open(os.path.expanduser(file_path), "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [40]:
drs_id = 'crdc:72eacf56-f87d-4af4-8128-47a6897d1506'
drs_url = mr.getAccessURL(drs_id, 'gs')
download(drs_url, '/Users/forei/Downloads/72eacf56-f87d-4af4-8128-47a6897d1506.dcm')

In [41]:
mr.getObject(drs_id)

{'access_methods': [{'access_id': 'gs',
   'access_url': {'url': 'gs://idc-open/72eacf56-f87d-4af4-8128-47a6897d1506.dcm'},
   'region': '',
   'type': 'gs'}],
 'aliases': [],
 'checksums': [{'checksum': 'da66bb0bccad300977e2c16762777868',
   'type': 'md5'}],
 'contents': [],
 'created_time': '2020-09-18T05:21:01.520509',
 'description': None,
 'form': 'object',
 'id': 'dg.4DFC/72eacf56-f87d-4af4-8128-47a6897d1506',
 'mime_type': 'application/json',
 'name': None,
 'self_uri': 'drs://nci-crdc.datacommons.io/dg.4DFC/72eacf56-f87d-4af4-8128-47a6897d1506',
 'size': 527968,
 'updated_time': '2021-11-15T19:04:09.450068',
 'version': 'f2030c4c'}

What if we do this across the whole study?


In [ ]:
q1 = Q('id = "TCGA-13-1504"')
r = q1.run()
print(r)

### TCGA-06-0118
Have previously downloaded dicom files from NBIA for this patient.

In [75]:
q1 = Q('subject_associated_project = "TCGA-OV"')
r = q1.run()
print(r)

Getting results from database



ERROR:root:
            Http Status: 500
            Error Message: No matching signature for operator = for argument types: ARRAY<STRING>, STRING. Supported signature: ANY = ANY at [1:72]
            


Total execution time: 880 ms
None


In [73]:
q1 = Q('ResearchSubject.associated_project = "TCGA-OV"')
r = q1.run()
print(r)


Getting results from database

Total execution time: 24808 ms

QueryID: baa7ae6d-bc80-4582-ac24-e132d6752a5d
Query: SELECT all_v2.* FROM gdc-bq-sample.integration.all_v2 AS all_v2, UNNEST(ResearchSubject) AS _ResearchSubject WHERE (_ResearchSubject.associated_project = 'TCGA-OV')
Offset: 0
Count: 100
Total Row Count: 608
More pages: True



In [99]:
query1 = """SELECT su.id sid , rs.identifier rsid, fi.drs_uri
from gdc-bq-sample.integration.all_v2 AS su,
unnest(ResearchSubject) AS rs,
unnest(rs.File) as fi
where (su.id = 'TCGA-13-1504')
and file_format = 'BAM' """

r1 = Q.sql(query1)
r1


QueryID: b59500f3-cf74-4e51-abf8-a659b12cc873
Query: SELECT su.id sid , rs.identifier rsid, fi.drs_uri
from gdc-bq-sample.integration.all_v2 AS su,
unnest(ResearchSubject) AS rs,
unnest(rs.File) as fi
where (su.id = 'TCGA-13-1504')
and file_format = 'BAM' 
Offset: 0
Count: 3
Total Row Count: 3
More pages: False

In [100]:
for res in r1:
    print(json.dumps(res, indent=3))

{
   "sid": "TCGA-13-1504",
   "rsid": [
      {
         "system": "GDC",
         "value": "cd49126a-ec15-43fa-9e43-3f7460d43f2b"
      }
   ],
   "drs_uri": "drs://dg.4DFC:8b7859ed-395a-449d-8e4b-7d385d0ffa43"
}
{
   "sid": "TCGA-13-1504",
   "rsid": [
      {
         "system": "GDC",
         "value": "cd49126a-ec15-43fa-9e43-3f7460d43f2b"
      }
   ],
   "drs_uri": "drs://dg.4DFC:698994fe-b22a-4723-b9f0-23551fd3bb0e"
}
{
   "sid": "TCGA-13-1504",
   "rsid": [
      {
         "system": "GDC",
         "value": "cd49126a-ec15-43fa-9e43-3f7460d43f2b"
      }
   ],
   "drs_uri": "drs://dg.4DFC:07a9e595-023d-47d5-97fa-3fcc62f5d0c6"
}


In [101]:
query2 = """SELECT su.id sid , rs.identifier rsid, fi.*
from gdc-bq-sample.integration.all_v2 AS su,
unnest(ResearchSubject) AS rs,
unnest(rs.File) as fi
where (su.id = 'TCGA-13-1504')
and file_format = 'BAM' """

r2 = Q.sql(query2)
r2


QueryID: b92d262a-8b86-4f1e-ada7-61123750661a
Query: SELECT su.id sid , rs.identifier rsid, fi.*
from gdc-bq-sample.integration.all_v2 AS su,
unnest(ResearchSubject) AS rs,
unnest(rs.File) as fi
where (su.id = 'TCGA-13-1504')
and file_format = 'BAM' 
Offset: 0
Count: 3
Total Row Count: 3
More pages: False

In [102]:
for res in r2:
    print(json.dumps(res, indent=3))

{
   "sid": "TCGA-13-1504",
   "rsid": [
      {
         "system": "GDC",
         "value": "cd49126a-ec15-43fa-9e43-3f7460d43f2b"
      }
   ],
   "id": "8b7859ed-395a-449d-8e4b-7d385d0ffa43",
   "identifier": [
      {
         "system": "GDC",
         "value": "8b7859ed-395a-449d-8e4b-7d385d0ffa43"
      }
   ],
   "label": "TCGA-13-1504-01A-01R-1565-13_mirna_gdc_realn.bam",
   "data_category": "Sequencing Reads",
   "data_type": "Aligned Reads",
   "file_format": "BAM",
   "associated_project": "TCGA-OV",
   "drs_uri": "drs://dg.4DFC:8b7859ed-395a-449d-8e4b-7d385d0ffa43",
   "byte_size": "617903851",
   "checksum": "42b51bf61122b716cd02a628da9b7d89"
}
{
   "sid": "TCGA-13-1504",
   "rsid": [
      {
         "system": "GDC",
         "value": "cd49126a-ec15-43fa-9e43-3f7460d43f2b"
      }
   ],
   "id": "698994fe-b22a-4723-b9f0-23551fd3bb0e",
   "identifier": [
      {
         "system": "GDC",
         "value": "698994fe-b22a-4723-b9f0-23551fd3bb0e"
      }
   ],
   "label": "C2